In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets
import collections
import numpy as np

## task 02

Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [2]:
# класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [3]:
# класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
   
        prediction = np.mean(self.labels)
        return prediction    

In [4]:
# дисперсия по выборке
def var(labels):

    return labels.std()**2

In [5]:
# Расчет качества

def quality(left_labels, right_labels, current_criteria, criteria):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
#     return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)
    return current_criteria - p * criteria(left_labels) - (1 - p) * criteria(right_labels)

In [6]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [7]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, criteria): 
    # минимальное количество объектов в узле
    min_leaf = 5
    
    current_criteria = criteria(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее min_leaf объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_criteria, criteria)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [8]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, criteria, max_depth, depth=0):
    if depth < max_depth: # критерий остановки по максимальной глубине дерева
        quality, t, index = find_best_split(data, labels, criteria)

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
        depth +=1
        # Рекурсивно строим два поддерева
        true_branch = build_tree(true_data, true_labels, criteria, max_depth, depth)
        false_branch = build_tree(false_data, false_labels, criteria, max_depth, depth)
        

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    else:
        return Leaf(data, labels)

In [9]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [10]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [11]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [12]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [13]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_regression(n_samples=1000, 
                                              n_features = 2, 
                                              n_informative = 2, 
                                              n_targets = 1, 
                                              noise = 5, 
#                                               coef = True, 
                                              random_state = 2)

In [14]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [15]:
# Построим дерево по обучающей выборке
my_tree_reg = build_tree(train_data, train_labels, var, max_depth = 5)

In [16]:
# print_tree(my_tree_reg)

In [17]:
# Получим ответы для обучающей выборки 

train_answers_reg = predict(train_data, my_tree_reg)

In [18]:
# И получим ответы для тестовой выборки

answers_reg = predict(test_data, my_tree_reg)

In [19]:
# Точность на обучающей выборке
train_accuracy_reg = calc_mse(train_labels, train_answers_reg)
train_accuracy_reg

463.75692042517096

In [20]:
# Точность на тестовой выборке
test_accuracy_reg = calc_mse(test_labels, answers_reg)
test_accuracy_reg

721.2269271196762

In [21]:
from sklearn.metrics import r2_score
r2_score(train_labels, train_answers_reg), r2_score(test_labels, answers_reg)

(0.9465266944201165, 0.8969516588020108)